In [ ]:
import re

MATH_MODULE = __import__('math')
ALLOWED_FUNCTIONS = {name: getattr(MATH_MODULE, name) for name in dir(MATH_MODULE) if name[0] != '_'}
    
class Expression:
    def __init__(self, expression):
        simplified = re.sub("O\((.+)\)", "\\1", expression)
        simplified = simplified.replace("N", "n")
        self.expression = simplified
        
    def execute(self, n):
        expression_python = self.expression.replace('n', str(n))
        
        try:
            return int(eval(
                expression_python,
                {
                    "__builtins__" : None
                }, 
                ALLOWED_FUNCTIONS
            ))
        except OverflowError:
            raise OverflowError("Unable to evaluate '{}' due to its size.".format(expression_python))

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

def generate_plot(expression_normal, expression_quantum, x_values, output):
    values_normal = [expression_normal.execute(x) for x in x_values]
    values_quantum = [expression_quantum.execute(x) for x in x_values]
    
    fig, ax = plt.subplots()
    ax.plot(x_values, values_normal, 'ro', label='Standart')
    ax.plot(x_values, values_quantum, 'go', label='Quantum')

    ax.set(
        xlabel='Number of inputs N', 
        ylabel='o-notation',
        title='Quantum speedup factoring'
    )

    ax.grid()
    ax.legend()
    
    with output:
        plt.plot()
        
ui = {
    "output": widgets.Output(),
    "text_complexity_quantum": widgets.Text(
        value='O(n^3)',
        placeholder='Please enter O(...).',
        description='Quantum complexity:',
        disabled=False
    ),
    "text_complexity_normal": widgets.Text(
        value='O(2 ** (n*log(log(n))))',
        placeholder='Please enter O(...).',
        description='Classic complexity:',
        disabled=False
    ),
    "button_calculate": widgets.Button(
        description='Calculate',
        disabled=False,
        button_style='success',
        tooltip='Calculate value',
        icon='check'
    )
}

layout = widgets.HBox([
   widgets.VBox([
       ui['text_complexity_quantum'],
       ui['text_complexity_normal'],
       ui['button_calculate']
   ]),
    widgets.VBox([
        ui['output']
    ])
])

def on_button_clicked(_):
    expression_classic = Expression(ui['text_complexity_normal'].value)    
    expression_quantum = Expression(ui['text_complexity_quantum'].value)
    values = [2 ** i for i in range(1, 10)]
    generate_plot(expression_classic, expression_quantum, values, ui['output'])
    
ui["button_calculate"].on_click(on_button_clicked)
display(layout)